In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/optiver-trading-at-the-close/public_timeseries_testing_util.py
/kaggle/input/optiver-trading-at-the-close/train.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/sample_submission.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/revealed_targets.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv
/kaggle/input/optiver-trading-at-the-close/optiver2023/competition.cpython-310-x86_64-linux-gnu.so
/kaggle/input/optiver-trading-at-the-close/optiver2023/__init__.py


In [2]:
train = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/train.csv')
revealed_targets = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/example_test_files/revealed_targets.csv')
test = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv')
sample_submission = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/example_test_files/sample_submission.csv')

In [3]:
train

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.000000,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.000000,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.000000,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.000000,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.000000,-7.349849,0,0_0_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5237975,195,480,540,2440722.89,-1,1.000317,28280361.74,0.999734,0.999734,1.000317,32257.04,1.000434,319862.40,1.000328,2.310276,26454,480_540_195
5237976,196,480,540,349510.47,-1,1.000643,9187699.11,1.000129,1.000386,1.000643,205108.40,1.000900,93393.07,1.000819,-8.220077,26454,480_540_196
5237977,197,480,540,0.00,0,0.995789,12725436.10,0.995789,0.995789,0.995789,16790.66,0.995883,180038.32,0.995797,1.169443,26454,480_540_197
5237978,198,480,540,1000898.84,1,0.999210,94773271.05,0.999210,0.999210,0.998970,125631.72,0.999210,669893.00,0.999008,-1.540184,26454,480_540_198


Missing Data - CHECK

In [4]:
train.isna().sum()

stock_id                         0
date_id                          0
seconds_in_bucket                0
imbalance_size                 220
imbalance_buy_sell_flag          0
reference_price                220
matched_size                   220
far_price                  2894342
near_price                 2857180
bid_price                      220
bid_size                         0
ask_price                      220
ask_size                         0
wap                            220
target                          88
time_id                          0
row_id                           0
dtype: int64

imbalance_size - The amount unmatched at the current reference price (in USD).

METHOD 1: Forward Fill or Backward Fill: Fill gaps with the previous (or next) non-null value.

METHOD 2: Interpolation: Fill missing values with interpolated data based on nearby points.



In [5]:
train['imbalance_size'].fillna(method='ffill', inplace=True)
# train['imbalance_size'].interpolate(method='linear', inplace=True)

In [6]:
train['reference_price'].fillna((train['bid_price'] + train['ask_price']) / 2, inplace=True)


Matched Size - since there is a strong correlation between matched_size and reference_price, use a regression model to predict and fill the mactched_size based on the reference_price values.  

In [7]:
from sklearn.linear_model import LinearRegression

subset_train_matched = train[['matched_size', 'reference_price']]
train_df = subset_train_matched.dropna()
predict_subset = train[train['matched_size'].isna()]

if not predict_subset.empty:
    # Define the model
    model = LinearRegression()

    # Train the model
    X_train = train_df[['reference_price']]
    y_train = train_df['matched_size']
    model.fit(X_train, y_train)

    # Handle missing values in the prediction subset
    mean_value = X_train['reference_price'].mean()
    X_predict = predict_subset[['reference_price']].copy()
    X_predict['reference_price'].fillna(mean_value, inplace=True)

    # Predict the matched_size values
    predicted_values = model.predict(X_predict)

    # Assign the predicted values back to the main dataframe
    train.loc[predict_subset.index, 'matched_size'] = predicted_values
else:
    print("There are no missing values in 'matched_size'.")



far_price - The crossing price that will maximize the number of shares matched based on auction interest only. This calculation excludes continuous market orders.


Iterative imputation (also called Multiple Imputation by Chained Equations - MICE) is a method where each incomplete variable is modeled as a function of other variables, and the missing values are estimated multiple times.

In [8]:
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


In [9]:
near_far_missing = train[['far_price', 'near_price']]


In [10]:
imputer = IterativeImputer(max_iter=10, random_state=0)
df_imputed = imputer.fit_transform(near_far_missing)
df_imputed = pd.DataFrame(df_imputed, columns=near_far_missing.columns)



In [11]:
train[['far_price', 'near_price']] = df_imputed[['far_price', 'near_price']]


Even though bid_price and bid_size are highly correlated, further testing needs to be done to confirm if regression imputation is the right approach. For now, Time Series imputation should suffice.

In [12]:
train['bid_price'].fillna(method='ffill', inplace=True)  # Forward fill
# OR
#train['bid_price'].interpolate(method='linear', inplace=True)  # Linear interpolation


Same with ask_price.

In [13]:
train['ask_price'].fillna(method='ffill', inplace=True)  # Forward fill
# OR
#train['ask_price'].interpolate(method='linear', inplace=True)  # Linear interpolation


reference_price - The price at which paired shares are maximized, the imbalance is minimized and the distance from the bid-ask midpoint is minimized, in that order. Can also be thought of as being equal to the near price bounded between the best bid and ask price.


Midpoint of Bid-Ask Spread:

Since the reference_price is defined as being similar to the near price bounded between the best bid and ask price, one reasonable approach to fill missing values would be to use the midpoint of the bid and ask prices (if you have access to these columns).

In [14]:
train['reference_price'].fillna((train['bid_price'] + train['ask_price']) / 2, inplace=True)


wap - The weighted average price in the non-auction book.
𝐵𝑖𝑑𝑃𝑟𝑖𝑐𝑒∗𝐴𝑠𝑘𝑆𝑖𝑧𝑒+𝐴𝑠𝑘𝑃𝑟𝑖𝑐𝑒∗𝐵𝑖𝑑𝑆𝑖𝑧𝑒 / 𝐵𝑖𝑑𝑆𝑖𝑧𝑒+𝐴𝑠𝑘𝑆𝑖𝑧𝑒

In [15]:
train.isna().sum()

stock_id                     0
date_id                      0
seconds_in_bucket            0
imbalance_size               0
imbalance_buy_sell_flag      0
reference_price              0
matched_size                 0
far_price                    0
near_price                   0
bid_price                    0
bid_size                     0
ask_price                    0
ask_size                     0
wap                        220
target                      88
time_id                      0
row_id                       0
dtype: int64

In [16]:
train['wap'] = (train['bid_price']*train['ask_size'] + train['ask_price']*train['bid_price']) / (train['bid_size'] + train['ask_size'])


In [17]:
train.isna().sum()

stock_id                    0
date_id                     0
seconds_in_bucket           0
imbalance_size              0
imbalance_buy_sell_flag     0
reference_price             0
matched_size                0
far_price                   0
near_price                  0
bid_price                   0
bid_size                    0
ask_price                   0
ask_size                    0
wap                         0
target                     88
time_id                     0
row_id                      0
dtype: int64

#### Feature Engineering ####

#### Test Submission 1 - Using Rolling Window Statistics as Feature Engineering & Preprocessing Methods ####

In [18]:
import pandas as pd
import numpy as np

from itertools import groupby
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import gc
from itertools import combinations
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [19]:
median_vol = train.groupby('stock_id')['bid_size'].median() + train.groupby('stock_id')['ask_size'].median()
train['bid_size_std_30'] = train.groupby(['stock_id'], as_index=False)['bid_size'].rolling(30, min_periods=1).std().sort_index()['bid_size']
train['bid_size_std_10'] = train.groupby(['stock_id'], as_index=False)['bid_size'].rolling(10, min_periods=1).std().sort_index()['bid_size']
train['bid_size_std_5'] = train.groupby(['stock_id'], as_index=False)['bid_size'].rolling(5, min_periods=1).std().sort_index()['bid_size']
train['ask_size_std_30'] = train.groupby(['stock_id'], as_index=False)['ask_size'].rolling(30, min_periods=1).std().sort_index()['ask_size']
train['ask_size_std_10'] = train.groupby(['stock_id'], as_index=False)['ask_size'].rolling(10, min_periods=1).std().sort_index()['ask_size']
train['ask_size_std_5'] = train.groupby(['stock_id'], as_index=False)['ask_size'].rolling(5, min_periods=1).std().sort_index()['ask_size']

In [20]:
def feat_eng(df):

    cols = [c for c in df.columns if c not in ['row_id', 'time_id', 'data_id']]
    df = df[cols]
    df['bid_plus_ask_sizes'] = df['bid_size'] + df['ask_size']
    df['median_vol'] = df['stock_id'].map(median_vol.to_dict())
    df['high_volume'] = np.where(df['bid_plus_ask_sizes'] > df['median_vol'], 1, 0)
    df['high_volume_ratio'] = np.where(df['bid_plus_ask_sizes'] > df['median_vol'], 1, 0)*df['bid_plus_ask_sizes']/df['bid_plus_ask_sizes'].sum()
    df['imbalance_ratio'] = df['imbalance_size'] / df['matched_size']
    df['high_volume_imbalance_size'] = df['high_volume']*df['imbalance_size']
    df['low_volume_matched_size'] = df['high_volume']*df['matched_size']
    df['mid_price'] = (df['ask_price'] + df['bid_price'])/2
    df['wwap'] = df.eval('(bid_size*bid_price+ask_size*ask_price)/(bid_size+ask_size)')

    df['imb_s1'] = df.eval('(bid_size-ask_size)/(bid_size+ask_size)')
    df['imb_s2'] = df.eval('(imbalance_size-matched_size)/(matched_size+imbalance_size)')

    df['ask_x_size'] = df.eval('ask_size*ask_price')
    df['bid_x_size'] = df.eval('bid_size*bid_price')

    df['ask_minus_bid'] = df['ask_x_size'] - df['bid_x_size']

    df['mix_diff_price'] = df['ask_minus_bid'].div(df['bid_plus_ask_sizes'])

    df["bid_size_over_ask_size"] = df["bid_size"].div(df["ask_size"])
    df["bid_price_over_ask_price"] = df["bid_price"].div(df["ask_price"])

    prices = ['reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap']

    for c in combinations(prices, 2):

        df[f'{c[0]}_minus_{c[1]}'] = (df[f'{c[0]}'] - df[f'{c[1]}']).astype(np.float32)
        df[f'{c[0]}_times_{c[1]}'] = (df[f'{c[0]}'] * df[f'{c[1]}']).astype(np.float32)
        df[f'{c[0]}_{c[1]}_imb'] = df.eval(f'({c[0]}-{c[1]})/({c[0]}+{c[1]})')

    for c in combinations(prices, 3):

        max_ = df[list(c)].max(axis=1)
        min_ = df[list(c)].min(axis=1)
        mid_ = df[list(c)].sum(axis=1)-min_-max_

        df[f'{c[0]}_{c[1]}_{c[2]}_imb2'] = (max_-mid_)/(mid_-min_)


    gc.collect()

    return df

In [21]:
%%time

y = train['target'].values
X = feat_eng(train.drop(columns='target'))

CPU times: user 34.8 s, sys: 5.18 s, total: 40 s
Wall time: 40 s


In [22]:
%%time

params = {'n_estimators': 10, 'learning_rate': 0.01, 'max_depth': 3, 'colsample_bytree': 0.95, 'min_child_weight': 31,
         'subsample_for_bin':500000, 'reg_alpha':0.01}
m = lgb.LGBMRegressor(**params, random_state=51)

m.fit(X, y)

CPU times: user 1min 59s, sys: 2.9 s, total: 2min 2s
Wall time: 36.1 s


LGBMRegressor(colsample_bytree=0.95, learning_rate=0.01, max_depth=3,
              min_child_weight=31, n_estimators=10, random_state=51,
              reg_alpha=0.01, subsample_for_bin=500000)

In [23]:
test.isna().sum()

stock_id                       0
date_id                        0
seconds_in_bucket              0
imbalance_size                 0
imbalance_buy_sell_flag        0
reference_price                0
matched_size                   0
far_price                  18229
near_price                 18000
bid_price                      0
bid_size                       0
ask_price                      0
ask_size                       0
wap                            0
time_id                        0
row_id                         0
dtype: int64

In [24]:
near_far_test_imp = test[['far_price', 'near_price']]
imputer_test = IterativeImputer(max_iter=10, random_state=0)
df_imputed_test = imputer_test.fit_transform(near_far_test_imp)
df_imputed_test = pd.DataFrame(df_imputed_test, columns=near_far_test_imp.columns)
test[['far_price', 'near_price']] = df_imputed_test[['far_price', 'near_price']]

In [25]:
test = feat_eng(test)

In [26]:
def zero_sum(prices, volumes):

#    I got this idea from https://github.com/gotoConversion/goto_conversion/

    std_error = np.sqrt(volumes)
    step = np.sum(prices)/np.sum(std_error)
    out = prices-std_error*step

    return out

In [27]:
import optiver2023
env = optiver2023.make_env()
iter_test = env.iter_test()

In [28]:
# Here is how I define rolling feature, however, it takes too much time to complete the test.
# It would be much helpful if anyone could improve the efficiency
stock_id = list(range(200))

def rolling_window(a, window):
    insert = np.full(window-1, a[0])
    a = np.insert(a, 0, insert)
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

def ts_stddev(data, window=10):
    value = rolling_window(np.array(data), window).std(axis=1)
    return value

def rolling_eng(test, test_all, counter):
    bid_size_std_60 = []
    bid_size_std_120 = []
    bid_size_std_300 = []
    ask_size_std_60 = []
    ask_size_std_120 = []
    ask_size_std_300 = []
    for sid in stock_id:
        bid_size = test_all[test_all['stock_id'] == sid]['bid_size']
        ask_size = test_all[test_all['stock_id'] == sid]['ask_size']
        bid_size_std_60.append(ts_stddev(bid_size, 30)[-1])
        bid_size_std_120.append(ts_stddev(bid_size, 10)[-1])
        bid_size_std_300.append(ts_stddev(bid_size, 5)[-1])
        ask_size_std_60.append(ts_stddev(ask_size, 30)[-1])
        ask_size_std_120.append(ts_stddev(ask_size, 10)[-1])
        ask_size_std_300.append(ts_stddev(ask_size, 5)[-1])
    test['bid_szie_std_30'] = np.array(bid_size_std_60)
    test['bid_size_std_10'] = np.array(bid_size_std_120)
    test['bid_size_std_50'] = np.array(bid_size_std_300)
    test['ask_size_std_30'] = np.array(ask_size_std_60)
    test['ask_size_std_10'] = np.array(ask_size_std_120)
    test['ask_size_std_5'] = np.array(ask_size_std_300)
    return test


In [29]:
counter = 0
for (test, revealed_targets, sample_prediction) in iter_test:
    test['counter'] = counter
#     print(test['counter'])
    if counter == 0:
        test_all = test
    else:
        test_all = pd.concat([test_all, test], axis = 0)

    test_roll = rolling_eng(test, test_all, counter)
    feat = feat_eng(test_roll)
    sample_prediction['target'] = m.predict(feat.drop(columns = ['counter']))
    sample_prediction['target'] = zero_sum(sample_prediction['target'], test.loc[:,'bid_size'] + test.loc[:,'ask_size'])
    env.predict(sample_prediction)
    counter += 1

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
